In [1]:
import string
# Библиотека построения индекса приближенного поиска ближайших соседей
import annoy
import numpy as np
import pandas as pd

from pymorphy3 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

In [2]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [12]:
# Загрузим текстовые описания товаров и преобразуем к нормализованному виду
shop_data = pd.read_csv("./data/ProductsDataset.csv")
shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))

shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [16]:
# Обучим модель fasttext на текстах
sentences = [i for i in shop_data['text'] if len(i) > 2]
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

In [22]:
# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_shop = annoy.AnnoyIndex(20 ,'angular')

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    
    # Инициализируется переменная "n_ft" со значением 0, которая будет
    # отслеживать количество слов, для которых были найдены эмбеддинги.
    n_ft = 0

    # В словарь "index_map_shop" добавляется соответствие между текущим
    # дентификатором айтема и его названием и ссылками на изображения.
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])

    # Создается вектор "vector_ft" размером 20, который инициализируется нулями.
    # В этом векторе будут суммироваться эмбеддинги слов для данного айтема.
    vector_ft = np.zeros(20)
    
    # Выполняется вложенный цикл по каждому слову в поле "text" текущего айтема. 
    for word in shop_data.loc[i, "text"]:
        # Для каждого слова проверяется, есть ли его эмбеддинг в модели
        # "modelFT.wv".
        # Если эмбеддинг найден, он добавляется к вектору "vector_ft", а счетчик
        # "n_ft" увеличивается на 1.
        if word in modelFT.wv.key_to_index:
            vector_ft += modelFT.wv.get_vector(word)
            n_ft += 1

    # Если хотя бы для одного слова был найден эмбеддинг, вектор "vector_ft"
    # нормализуется путем деления на значение "n_ft".
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
        
    # Айтем добавляется в индекс "ft_index_shop" с помощью метода "add_item" с
    # использованием текущего значения счетчика "counter" и вектора "vector_ft".
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

# После завершения цикла индекс "ft_index_shop" строится с использованием метода "build". Параметр "10" указывает на количество деревьев, которые будут использованы для построения индекса. Чем больше деревьев, тем точнее будет поиск, но и требуется больше памяти и времени для построения индекса.
ft_index_shop.build(10)

/var/folders/df/dz8wtw0j7791hg98mrx_thxc0000gn/T/ipykernel_71169/3635255799.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(shop_data))):


  0%|          | 0/35548 [00:00<?, ?it/s]

True

In [9]:
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [24]:
# Попробуем порекомендовать что-то
# Так как у нас нет пользовательских сессий, то оценка качества "На глаз"
def recommend(item_id):
    # Получим тайтл айтема по идентификатору
    title = shop_data[shop_data["product_id"] == item_id]["title"].values[0]
    title = preprocess_txt(title)

    # Создание нулевого вектора vector_ft размером 20 и инициализация счетчика n_ft с нулевым значением.
    # Вектор vector_ft будет использоваться для суммирования эмбеддингов слов айтема.
    vector_ft = np.zeros(20)

    # Счетчик n_ft будет отслеживать количество слов, для которых были найдены эмбеддинги.
    n_ft = 0
    # Каждое слово обернем в эмбеддинг
    # Обработка каждого слова из поля "text" айтема:
    # Запускается цикл, который перебирает каждое слово из строки, предположительно, содержащей текст айтема.
    for word in shop_data.loc[i, "text"]:
        # Для каждого слова проверяется, есть ли его эмбеддинг в модели modelFT.wv.
        # Если эмбеддинг найден, он добавляется к вектору vector_ft.
        # Также увеличивается значение счетчика n_ft на 1.
        if word in modelFT.wv.key_to_index:
            vector_ft += modelFT.wv.get_vector(word)
            n_ft += 1

    # Если значение счетчика n_ft больше 0 (то есть были найдены эмбеддинги для каких-то слов),
    # то вектор vector_ft делится на n_ft для нормализации.
    if n_ft > 0:
        vector_ft = vector_ft / n_ft

    # Поиск наиболее близких айтемов похожих по названию:
    # Используется метод get_nns_by_vector объекта ft_index_shop, 
    # который возвращает индексы ближайших айтемов в индексе ft_index_shop на основе переданного вектора.
    # Индексы ближайших айтемов сохраняются в переменной rec_items.
    rec_items = ft_index_shop.get_nns_by_vector(vector_ft, 5)
    
    # Составление списка названий рекомендуемых айтемов:
    # Создается список, в который добавляются названия айтемов, соответствующие индексам в rec_items.
    # Названия айтемов извлекаются из словаря index_map_shop, где ключами являются идентификаторы айтемов.
    return [index_map_shop[i][0] for i in rec_items]


recommend("58e3cfe6132ca50e053f5f82")

['Пиджак мужской 46-48',
 'Платья',
 'Часы',
 'Шлема, защита голени, перчатки',
 'Блузка']